In [21]:
import urllib.request
import json
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
from textblob import TextBlob
import nltk
import plotly.graph_objs as go
import time 
from google.colab import drive
from google.colab import files

nltk.download('punkt')
url = "https://www.aljazeera.com/where/mozambique/"
path = '/content/drive'
drive.mount(path)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
pip freeze > requirements.txt

In [33]:
files.download('requirements.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
def crawl(url):
  try:
    response = urllib.request.urlopen(url,timeout=3)
    h = response.read()
    h = h.decode('utf-8')
    return h
  except: 
    print('Invalid url or time out')
    return None

In [5]:
def parse(html,url=None):
  soup = BeautifulSoup(html,"html.parser")
  out_urls = soup.find_all('a',{'class':'u-clickable-card__link'})
  out_urls_list = []
  for single_url in out_urls:
    href ="https://www.aljazeera.com"+single_url['href']
    out_urls_list.append(href)
  
  return out_urls_list


In [28]:
def collect_and_save_to_json(url_list):
  article_text_dict = {}
  for i in tqdm(range(10)): #the articles from the scraper are automatically ordered by their publicaiton dates, so just take the first 10. 
    
    html = crawl(url_list[i])
    soup = BeautifulSoup(html,"html.parser")
    content = soup.find_all('p')
    final_text = ""

    for c in content:
      final_text +=" "+c.text

    article_text_dict[i]=final_text

  with open('article_compilation.json', 'w') as json_file:
    json.dump(article_text_dict,json_file)

  files.download('article_compilation.json')

  return article_text_dict
      


In [7]:
def sentiment_analysis(article_text_dict):
  sentiment_result = []
  for i in article_text_dict:
    blob=TextBlob(article_text_dict[i])
    sentiment_result.append(blob.sentiment)
  return sentiment_result

  

In [8]:
def plot(sentiment_result):
  
  polarity =[]
  subjectivity = []
  for i in sentiment_result: 
    polarity.append(i.polarity)
  fig = go.Figure(
    data=[go.Bar(y=polarity)],
    layout_title_text="Sentiment for the latest 10 news articles"
  )
  fig.show()

  for i in sentiment_result: 
    subjectivity.append(i.subjectivity)
  fig2 = go.Figure(
    data=[go.Bar(y=subjectivity)],
    layout_title_text="Subjectivity for the latest 10 news articles"
  )
  fig2.show()



  return


In [29]:
def main():
    start = time.time()
    
    main_page_info = crawl(url)
    url_list = parse(main_page_info,url)
    article_text_dict = collect_and_save_to_json(url_list)
    sentiment_result = sentiment_analysis(article_text_dict)
    plot(sentiment_result)

    end = time.time()
    print("time taken: ",end - start)


if __name__=="__main__":
    main()

100%|██████████| 10/10 [00:10<00:00,  1.05s/it]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

time taken:  11.028868198394775
